# Chrome Dino re-write 2023

Use Pytorch, stable-baselines3, etc. to implement an agent to play chrome://dino with reinforcement learning.

## dependencies
- CUDA-enabled Pytorch (Pytorch 2.00; CUDA 1.18)
- Stable-Baselines3 (with extras like OpenCV): https://stable-baselines3.readthedocs.io/en/master/
- Protobuf (a training dependency) (3.20.*)
- pytesseract (interface to Google Tesseract)
- Google Tesseract-OCR ((5.3.1.20230401)
- Gymn (gym v0.21 since this is used by Stable-Baselines3; RL environment library): https://gymnasium.farama.org/

In [ ]:
%%capture
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
%%capture
# needed to support old version of gym (0.21) used with Stable-Baselines3
# gym 0.21 has installation issue; gym moved to Gymnasium; SB3 still does not support Gymnasium 
!pip install setuptools==66 Cmake git+https://github.com/openai/gym.git@9180d12e1b66e7e2a1a622614f787a6ec147ac40

In [ ]:
%%capture
!pip install stable-baselines3[extra] protobuf==3.20.*

In [ ]:
%%capture
!pip install mss pydirectinput pytesseract

In [ ]:
import numpy as np
from gym import Env
from gym.spaces import Box, Discrete

## set up environment

In [ ]:
# create a base Gym environment for managing state
# Env must implement step and reset; optionally, render 
# https://www.gymlibrary.dev/api/core/
class ChromeDinoRL(Env):
    
    # initialize environment spaces
    def __init__(self):
        super().__init__()
        
        # set up observations and actions
        # multidimensional array as return output from observation (an image)
        self.observation_space = Box(low=0, high=255, shape=(1,83,100), dtype=np.uint8)
        # two actions: jump or not
        self.action_space = Discrete(2)
        
        
    # run one timestep in the environment
    def step(self, action):
        # 0 -> spacebar, 1 -> noop
        pass
        
        
    # reset to initial state, return initial observation
    def reset(self):
        pass
        
    # compute render frames for visualization; render with cv2
    def render(self):
        pass
        
    # do an observation
    def observe_environment(self):
        pass
        
    # identify game over
    def game_over(self):
        pass
    

In [ ]:
# test environment
env = ChromeDinoRL()

In [ ]:
env.action_space.sample()

## train model

Train the DQN and save it using a Stable-Baselines3 callback

In [ ]:
# training imports
from stable_baselines3 import DQN
from stablebaselines3.common.callbacks import BaseCallback
import os